<a href="https://colab.research.google.com/github/paulxdillon/Chem-Eng/blob/master/ResidueCurveMaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculation of azeotropes.

In [11]:
!pip install pyomo
import numpy as np
from scipy.optimize import curve_fit
import pylab
from scipy.optimize import fsolve
from scipy.integrate import odeint
import scipy as sp
from scipy.integrate import ode
from pyomo.environ import *
import pandas as pd

In [0]:
Components = ['Water','2-MCH','Acetone']
Pairs = ['Water/2-MCH', 'Water/Acetone', '2-MCH/Acetone']

In [0]:
NRTLparams = {'Pairs':['Water/2-MCH','Water/Acetone','2-MCH/Acetone','IPA/Ethanol','IPA/Methanol','Ethanol/Methanol','Water/Ethanol', 'Water/Methanol', 'IPA/Water',],
              'aij': [5.06045,      0.0544,         -1.01139,            0,              0,              0,              0,              0,                 -1.3115],
              'aji': [-2.4211,      6.3981,         1.28690,             0,              0,              0,              0,              0,                 6.8284],
              'bij': [-34.9329,     419.972,       115.763,            -266.3770,      -407.2343,      6.4080,         670.444,        425.323,             426.398],
              'bji': [1213.92,      -1808.99,        -112.44,            347.2905,       603.0893,       -13.0821,        -55.1698,       -127.757,          -1483.5],
              'cij': [0.2966,       0.3,            0.2992,             0.3125,         0.3012,         0.3356,         0.3031,         0.3012,               0.3]}

In [0]:
pair12 = Components[0]+'/'+ Components[1] #'Water/2-MCH'
pair13 = Components[0]+'/'+ Components[2] #'Water/Acetone'
pair23 = Components[1]+'/'+ Components[2] #'2-MCH/Acetone'

In [15]:
print(pair12, pair13, pair23)

Water/2-MCH Water/Acetone 2-MCH/Acetone


In [0]:
ncomp = 3
tau = np.ndarray((ncomp, ncomp),float)
alpha = np.ndarray((ncomp, ncomp),float)

df = pd.DataFrame(NRTLparams)
df.set_index("Pairs", inplace=True)

df.loc[pair12]
df.loc[pair12].loc['aij']

P = 1.01325 # barA
T = 60      # deg C (preliminary value)
a12 = df.loc[pair12].loc['aij']
a21 = df.loc[pair12].loc['aji']

a13 = df.loc[pair13].loc['aij']
a31 = df.loc[pair13].loc['aji']

a23 = df.loc[pair23].loc['aij']
a32 = df.loc[pair23].loc['aji']

b12 = df.loc[pair12].loc['bij']
b21 = df.loc[pair12].loc['bji']

b13 = df.loc[pair13].loc['bij']
b31 = df.loc[pair13].loc['bji']

b23 = df.loc[pair23].loc['bij']
b32 = df.loc[pair23].loc['bji']

c12 = df.loc[pair12].loc['cij']
c21 = df.loc[pair12].loc['cij']
c13 = df.loc[pair13].loc['cij']
c31 = df.loc[pair13].loc['cij']
c23 = df.loc[pair23].loc['cij']
c32 = df.loc[pair23].loc['cij']

In [0]:
tau[0,0] = 0
tau[0,1] = a12 +  b12/(T+273.15)
tau[0,2] = a13 +  b13/(T+273.15)

tau[1,0] = a21 +  b21/(T+273.15)
tau[1,1] = 0
tau[1,2] = a23 +  b23/(T+273.15)

tau[2,0] = a31 +  b31/(T+273.15)
tau[2,1] = a32 +  b32/(T+273.15)
tau[2,2] = 0

alpha[0,0] = 0
alpha[0,1] = c12
alpha[0,2] = c13

alpha[1,0] = c12
alpha[1,1] = 0
alpha[1,2] = c23

alpha[2,0] = c13
alpha[2,1] = c23
alpha[2,2] = 0

In [0]:
def nrtl(alpha, tau, t, x):
 
    G = np.exp(-alpha*tau)
    ncomp = x.shape[0]
    gamma = np.zeros_like(x)
    summ = 0
    
    for i in range(ncomp):
        summ = 0
        for j in range(ncomp):
            summ += x[j]*G[i,j]/np.sum(G[:,j]*x)*(tau[i,j] - (np.sum(x*tau[:,j]*G[:,j])/np.sum(G[:,j]*x)))
        gamma[i] = np.sum(tau[:,i]*G[:,i]*x)/np.sum(G[:,i]*x) + summ    
        
    return np.exp(gamma)

In [0]:
def gMfn(alpha, tau, t, x):
    G = np.exp(-alpha*tau)
    ncomp = x.shape[0]
    summ = 0
    sumi = 0
    gE_RT = 0
    gMix_RT = 0

    for i in range(ncomp):
       summ = 0
       sumj = 0
       sumk = 0
       for j in range(ncomp):
           sumj = sumj + tau[j,i]*G[j,i]*x[j];

       for k in range(ncomp):
           sumk = sumk + G[k,i]*x[k]

       sumi = sumi + x[i]*sumj/sumk

    return sumi*8.314*(t+273.15)

In [20]:
print(df)

                      aij     aji       bij        bji     cij
Pairs                                                         
Water/2-MCH       5.06045 -2.4211  -34.9329  1213.9200  0.2966
Water/Acetone     0.05440  6.3981  419.9720 -1808.9900  0.3000
2-MCH/Acetone    -1.01139  1.2869  115.7630  -112.4400  0.2992
IPA/Ethanol       0.00000  0.0000 -266.3770   347.2905  0.3125
IPA/Methanol      0.00000  0.0000 -407.2343   603.0893  0.3012
Ethanol/Methanol  0.00000  0.0000    6.4080   -13.0821  0.3356
Water/Ethanol     0.00000  0.0000  670.4440   -55.1698  0.3031
Water/Methanol    0.00000  0.0000  425.3230  -127.7570  0.3012
IPA/Water        -1.31150  6.8284  426.3980 -1483.5000  0.3000
